In [1]:
# %pip install --upgrade pip
# %pip install fairseq2 sonar-space
# %pip install libsndfile

In [2]:
import torch

from Encoders.text.SONAR import SONAR as SONAR_ENC
from Encoders.text.SONAR import SONAR as SONAR_DEC
from Encoders.text.BART import BART as BART_ENC
from Decoders.text.BART import BART as BART_DEC

In [3]:
input_string = ['Greetings fellow traveler']
encoder = SONAR_ENC()
decoder = SONAR_DEC()

parameter load: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   387/387 100% 0:00:00

In [4]:
semantic_vector = encoder.encode(input_string)

/opt/anaconda3/lib/python3.12/site-packages/torch/__init__.py:1551: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Context.cpp:85.)
  return _C._get_float32_matmul_precision()


In [5]:
outputs = []
for _ in range(10):
    reconstructed = decoder.decode(semantic_vector)
    outputs.append(reconstructed)

print(outputs)


AttributeError: 'SONAR' object has no attribute 'decode'